# Code for preprocessing of a graph dataset

## FORMAT of INPUT and OUTPUT files 
 
- INPUT FILE FORMAT: 
 
    ``` 
    #nodes 
    node01 node02 weight 
    ``` 
 
    ... 
 
- OUTPUT FILE FORMAT: 
 
    ``` 
    #nodes 
    node01-1 node02-1 weight 
    ``` 
 
    ...

### For every forward edge A-B, cancel the corresponding backward edge B-A (if it exists):

In [1]:
import networkx as nx
import matplotlib.pyplot as plt
from networkx.drawing.nx_agraph import to_agraph
import random

In [2]:
inputFileName = "../inputs/airports_8000.txt" 
# inputFileName = "../inputs/airports_1500.txt" 
outputFileName = "../inputs/airports_8000_directed.txt" 
# outputFileName = "../inputs/airports_1500_directed.txt" 
 
# inputFileNameDirected =  outputFileName 
inputFileNameDirected =  "../inputs/airports_8000_directed_normalized.txt" 
# outputFileNameDAG = "../inputs/airports_8000_dag.txt" 
outputFileNameDAG = "../dag.txt" 
# outputFileNameDAG = "../inputs/airports_1500_dag.txt" 
# n = 10000= 100000


In [108]:
# function to check the number of nodes (labels of nodes are not consecutive)
def countNodes(fileName):
    fin = open(fileName, "r")
    numNodes = fin.readline().strip()
    
    # lst = [" " for i in range(int(numNodes))]
    lst = [" " for i in range(n)]

    for line in fin:
        vals = line.strip().split()
        lst[int(vals[0])-1] = "*"
        lst[int(vals[1])-1] = "*"

    cnt = 0
    for i in range(len(lst)):
        if lst[i] == "*":
            cnt +=1
    print(f"Number of nodes in graph from {fileName}: ", cnt)

In [109]:
# for every edge A-B, remove backward edge B-A (if it exist)
fin = open(inputFileName, "r")
fout = open(outputFileName, "w")

countNodes(inputFileName)
d = dict()
fout.write("                  \n")

first = fin.readline()
for line in fin:
    val = line.strip().split()
    # check if key in dictionary and if val in list of key 
    if not (val[1] in d and val[0] in d[val[1]]):
        fout.write(f"{int(val[0]) - 1} {int(val[1]) - 1} {int(val[2])}\n")        
        if (val[0] not in d):   
            d[val[0]] = []
        d[val[0]].append(val[1])

fin.close()
fout.close()

fout = open(outputFileName, "r+")
fout.write(f"{len(d)}")
print(d.keys())
fout.flush()

fout.close()
countNodes(outputFileName)



- Node enumeration starting from 1

input file: nodes start from 0

output file: nodes start from 1
* useful to generate input file with dataset to be tested with Matlab's max flow algorithm

In [ ]:
# generate output file with node values from 1 onwards

fin = open(inputFileName, "r")
fout = open(outputFileName, "w")

line = fin.readline()
fout.write(line)

for line in fin:
    vals = line.split()
    vals[0] = int(vals[0]) + 1
    vals[1] = int(vals[1]) + 1
    fout.write(str(vals[0]) + " " + str(vals[1]) + " " + vals[2] + "\n")

fin.close()
fout.close()

Clear cycles from the graph

In [5]:
def graph_from_file(filename):
    G = nx.DiGraph()    # G is a directed graph
    with open(filename) as f:
        line = f.readline()
        for line in f:
            node1, node2, weight = line.split()
            G.add_edge(node1, node2, capacity=int(weight))
    return G

In [ ]:
def remove_cycle(G):
    # Find a cycle in the graph
    cycle = nx.find_cycle(G)
    # print(f"Cycle found: {cycle}")
    
    # Remove one edge from the cycle to break it
    node1 = cycle[0][0]
    node2 = cycle[0][1]
    G.remove_edge(node1, node2)
    # print(f"Removed edge: {node1}, {node2}")

    return G

def main():

    G = graph_from_file(inputFileNameDirected)
    fout = open(outputFileNameDAG, "w")
       
    # check if the directed graph is acyclic, i.e. if it is a DAG
    isDag = nx.is_directed_acyclic_graph(G)
    print("The graph is DAG:", isDag)
    print("Number of edges:", G.number_of_edges())
    print("Number of nodes:", G.number_of_nodes())
    countNodes(inputFileNameDirected)

    # if not acyclic (i.e. there is at least 1 cycle), remove all cycle(s)
    while not isDag:
        G = remove_cycle(G)
        isDag = nx.is_directed_acyclic_graph(G)

    print("The graph is DAG:", isDag)
    print("Number of edges:", G.number_of_edges())
    print("Number of nodes:", G.number_of_nodes())
    fout.write(f"{G.number_of_nodes()}\n")
    # print(f"{G.number_of_nodes()}")
    for edge in G.edges():
         fout.write(f"{edge[0]} {edge[1]} {G[edge[0]][edge[1]]['capacity']}\n")
        # print(f"{edge[0]} {edge[1]} {G[edge[0]][edge[1]]['capacity']}\n")
    
    fout.close()
    countNodes(outputFileNameDAG)
    
main()

In [44]:
graph = graph_from_file(outputFileNameDAG)
source = "46"
sink = "0"

### check if path exists between source and sink

In [10]:
def is_path_between_source_sink(g, src, snk):
    return bool(nx.has_path(g, src, snk))

### check if source has only outgoing edges

In [4]:
def can_be_source(g, s):
    incoming_edges = list(g.in_edges(s)) # returns all edges with provided node as target node
                             # if there are no edges incoming in the node, it can be a source
    if not incoming_edges:
        return True
    return False

### check if sink has only incoming edges

In [8]:
def can_be_sink(g, s):
    outgoing_edges = list(g.edges(s)) # finds all the edges with provided node as source
    if not outgoing_edges:                    # if there are no edges, the node can be a sink
        return True
    return False

### Find maximum flow of certain graph

In [15]:
graph = graph_from_file(outputFileNameDAG)

In [17]:
source = "0"
sink = "6999"
is_source = can_be_source(graph, source)
is_sink = can_be_sink(graph, sink)
print(f"{source} can be source? {is_source}")
print(f"{sink} can be sink? {is_sink}")
if is_source and is_sink:
    print(f"Is there a path between {source} and {sink}? {'Yes :D' if is_path_between_source_sink(graph, source, sink) else 'No :('}")


0 can be source? True
6999 can be sink? True
Is there a path between 0 and 6999? Yes :D


### Compute maximum flow between source and sink

In [18]:
flow = nx.maximum_flow(graph,source,sink)[0]
print(f"Maximum flow between {source} and {sink}: {str(flow)}")

Maximum flow between 0 and 6999: 1


In [14]:
lst_sources = []
lst_sinks = []
lst_paths = []
sk = graph.number_of_nodes()
for i in range(0,sk,1):
    if can_be_source(graph, str(i)):
        lst_sources.append(str(i))
for i in range(sk, -1, -1):
    if can_be_sink(graph, str(i)):
        lst_sinks.append(str(i))
for source in lst_sources: 
    for sink in lst_sinks:
        if is_path_between_source_sink(graph, source, sink):
            lst_paths.append((source, sink))
print(f"Found {len(lst_paths)} valid paths")

Found 156204 valid paths


In [ ]:
for (source, sink) in lst_paths:
    if source != sink:
        flow = nx.maximum_flow(graph,source,sink)[0]
        if flow > 1:
            print(f"Maximum flow between {source} and {sink}: {str(flow)}")

NORMALIZE NOTATION OF NODES IN A FILE

In a graph with N nodes with random non-consecutive labels, after the preprocessing, have all nodes with consecutove labels:
- start from node 0
- all nodes have consecutive labels
- the last node has a label=N-1

In [104]:
# ifN = "../inputs/airports_1500_dag.txt"
ifN = "../inputs/airports_8000_directed.txt"
# ofN = "../inputs/airports_1500_dag_normalized.txt"
ofN = "../inputs/airports_8000_directed_normalized.txt"

def readList(fName):
    fin = open(ifN, "r")
    first = fin.readline()
    lst = [-1 for i in range(10000)]
    for line in fin:
        vals = line.strip().split()
        lst[int(vals[0])] = int(vals[0])
        lst[int(vals[1])] = int(vals[1])
    fin.close()
    return lst

lst = readList(ifN)
cnt=0
for i in range(10000):
    if lst[i] != -1:
        cnt+=1
print("# nodes:", cnt)

lab = 0
for i in range(10000):
    if lst[i] != -1:
        lst[i] = lab
        lab+=1

fin = open(ifN, "r")
fout = open(ofN, "w")
fout.write(f"{cnt}\n")
first = fin.readline()
for line in fin:
    vals = line.strip().split()
    # print(lst[int(vals[0])-1], lst[int(vals[1])-1], int(vals[2]))
    fout.write(f"{lst[int(vals[0])]} {lst[int(vals[1])]} {int(vals[2])}\n")
fin.close()
fout.close()

fin = open(ofN, "r")
n_nodes = int(fin.readline())
lst3 = ["*" for i in range(n_nodes)]
for line in fin:
    vals = line.strip().split()
    lst3[int(vals[0])] = "ok"
    lst3[int(vals[1])] = "ok"
fin.close()

ok = True
i=0
while i<cnt and ok:
    if lst3[i] != "ok":
        ok = False
    else:
        i+=1
if ok:
    print(f"ok, index i = {i}")
else:
    print(f"not ok, index i = {i}")
# print(lst3)

# nodes: 2939
ok, index i = 2939
